In [2]:
import numpy as np
import pandas as pd
import json
from geopy.distance import great_circle

Read json file contains border points coordinates:

In [3]:
with open("Border_Points.json", "r") as file:
    data = json.load(file)

In [4]:
data = data["features"][0]["geometry"]["coordinates"][0]
len(data)

2284

To obtain destribution of points in each of four coordinate regions:

In [5]:
Sari_coor   = [36.561151, 53.060088] # representing a point separates Iranian border into eastern and wertern regions
Ilam_coor = [33.568375, 46.045389]  # represinting a point separates Iranian border into northern and southern regions

In [6]:
Ilam_lat = Ilam_coor[0]
Ilam_lng = Ilam_coor[1]
Sari_lat = Sari_coor[0]
Sari_lng = Sari_coor[1]

In [7]:
points = pd.DataFrame(data, columns=["lng", "lat"])
points.head()

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005


In [8]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 94
len_NW : 446
len_SW : 768
len_SE : 976


To complete points in NE region:

In [9]:
with open("NE_Complete.json", "r") as file:
    NE_Completoin_data = json.load(file)

In [10]:
NE_Completion = list()

for feature in NE_Completoin_data["features"]:
    NE_Completion.append(feature["geometry"]["coordinates"][:])

NE_Completion = [element for sublist in NE_Completion for element in sublist]

In [11]:
NE_Completion = pd.DataFrame(NE_Completion, columns=["lng", "lat"])

In [12]:
len(NE_Completion)

923

In [13]:
NE_Completion.head()

,lng,lat
0,60.551479,33.640504
1,60.551331,33.643335
2,60.551035,33.645919
3,60.551183,33.648627
4,60.551183,33.651580


To complete points in NW region:

In [14]:
with open("NW_Complete.json", "r") as file:
    NW_Completoin_data = json.load(file)

In [15]:
NW_Completion = list()

for feature in NW_Completoin_data["features"]:
    NW_Completion.append(feature["geometry"]["coordinates"][:])

NW_Completion = [element for sublist in NW_Completion for element in sublist]

In [16]:
NW_Completion = pd.DataFrame(NW_Completion, columns=["lng", "lat"])

In [17]:
len(NW_Completion)

552

In [18]:
NW_Completion.head()

,lng,lat
0,45.924524,33.598682
1,45.912369,33.620953
2,45.902645,33.641194
3,45.844300,33.627025
4,45.810265,33.610830


To complete points in SW region:

In [19]:
with open("SW_Complete.json", "r") as file:
    SW_Completoin_data = json.load(file)

In [20]:
SW_Completion = list()

for feature in SW_Completoin_data["features"]:
    SW_Completion.append(feature["geometry"]["coordinates"][:])

SW_Completion = [element for sublist in SW_Completion for element in sublist]

In [21]:
SW_Completion = pd.DataFrame(SW_Completion, columns=["lng", "lat"])

In [22]:
len(SW_Completion)

216

In [23]:
SW_Completion.head()

,lng,lat
0,54.007617,26.755803
1,53.954357,26.735419
2,53.885879,26.708235
3,53.802184,26.721828
4,53.741315,26.701438


To merge dataframes:

In [24]:
points = pd.concat([points, NE_Completion, NW_Completion, SW_Completion], ignore_index=True)
points

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005
...,...,...
3970,46.169003,33.069856
3971,45.866036,33.496451
3972,45.946185,33.548421
3973,45.913532,33.630024


Obtain destribution of points in each of four coordinate regions one more time:

In [25]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 1017
len_NW : 1000
len_SW : 963
len_SE : 995


Optimization and searching for the exact center of Iran:

In [26]:
lng_min = 49.781853529617024
lng_max = 58.37602322566812
lat_min = 29.235933551806966
lat_max = 35.393730146031245

In [27]:
points_copy = points.copy()

In [38]:
center_candidates = []
lat_candidate = 0
lng_candidate = 0

In [39]:
step = 5
min_variance = 145231.6822812455

while step > 0.00000000000001:
    for lng in np.arange(lng_min, lng_max, step):
        for lat in np.arange(lat_min, lat_max, step):
            points["distances"] = points.apply(lambda row: great_circle((row["lat"], row["lng"]), (lat_min, lng_min)).kilometers, axis=1)        # calculate variance
            variance = points["distances"].var(ddof=0)
            if variance < min_variance:
                min_variance = variance
                lat_candidate = lat
                lng_candidate = lng
    center_candidates.append([lat_candidate, lng_candidate])
    step /= 10


NameError: name 'lat_candidate' is not defined

In [34]:
# points["distances"] = points.apply(lambda row: np.sqrt((row["lng"]-lng_min)**2 + (row["lat"]-lat_min)**2), axis=1)
points["distances"] = points.apply(lambda row: great_circle((row["lat"], row["lng"]), (lat_min, lng_min)).kilometers, axis=1)

In [34]:
points

,lng,lat,distances
0,48.613949,29.928497,1.357809
1,48.621193,29.934804,1.354826
2,48.624422,29.934589,1.351949
3,48.630881,29.932651,1.345419
4,48.640818,29.932005,1.336592
...,...,...,...
3970,46.169003,33.069856,5.267983
3971,45.866036,33.496451,5.786678
3972,45.946185,33.548421,5.771473
3973,45.913532,33.630024,5.854224


In [36]:
points["distances"].var(ddof=0)

145231.6822812455